# Using First-order LINE to embed Cora
In this tutorial, we will see how to use the First-order LINE
node embedding method to compute embedding of the Cora graph.
We will retrieve the graph, compute its report, and the get visualizations, both
single-plot, complete and animated.

We will show how to get visualizations both using the most immediate
approach, simply using the `GraphVisualizer` object with the embedding method
name, and the slightly longer approach creating the embedding model and
computing the embedding.

Of note, GRAPE provides multiple implementations of the First-order LINE 
model, using as backend the Ensmallen and TensorFlow libraries, and we will show how to use all of them.

## Installing the library
First of all, to install GRAPE just run as usual:

```bash
pip install grape
```

In [ ]:
!pip install -qU grape

## Retrieving the graph
To retrieve the Cora graph from the linqs graph repository
we run the following line of code:

In [ ]:
from grape.datasets.linqs import Cora
graph = Cora()

Do note that we support the retrieval of many other graphs, 83644 at the time of writing,
and you just need to import the graph you desire from the `datasets` submodule.

Remember that you can peruse the complete list of available graphs by running:

In [ ]:
from grape.datasets import get_all_available_graphs_dataframe
all_available_graphs_dataframe = get_all_available_graphs_dataframe(verbose=False)
all_available_graphs_dataframe

Now, in the retrieved version of the Cora graph we find both nodes representing the
papers and nodes representing words present in the paper abstracts.

In most benchmarks, only the topology represented by the paper nodes is used and therefore
we run a filter that splits the graph into the topology of exclusively the paper nodes and
a dataframe containing the one-hot encoded words vectors.

In [ ]:
from grape.datasets.linqs import get_words_data
graph, node_features = get_words_data(graph)

## Graph report
Let's proceed to compute the graph report, which includes both the ordinary topological
information, such as the number of edges and number of nodes, number of connectes components
and number of singletons, plus many topological oddities.

In [ ]:
graph

## Graph visualizations
In GRAPE we support the visualization of a wide variety of graph properties,
which may either visualized through PCA, TSNE or UMAP decompositions, as a single plot
per property or as a composite plot. Furthermore, single plots may animated through
a high-dimensional rotation that allows for a more comprensive view of the embedding
decompositions.

### Computing embedding and plotting the node types
We start by showing how to use the `GraphVisualizer` object to compute in a few lines of code
a provided node embedding method, and to use the TSNE decomposition to display the node
types both in 2D and 4D. The 4D animation is achieved by executing a 4D decomposition,
rotating the decomposition in 4D and then plot the first 3 dimensions in a 3-dimensional plot.

### Ensmallen-based model

In [ ]:
from grape.embedders import FirstOrderLINEEnsmallen
embedding = FirstOrderLINEEnsmallen().fit_transform(graph)

In [ ]:
from grape import GraphVisualizer
visualizer = GraphVisualizer(graph)

# You can either provide the model name
#visualizer.fit_nodes("First-order LINE", library_name="Ensmallen")
# Or provide a precomputed embedding
#
# visualizer.fit_nodes(numpy_array_with_embedding)
# visualizer.fit_nodes(pandas_dataframe_with_embedding)
#
# or alternatively provide the model to be used:
#
# from grape.embedders import FirstOrderLINEEnsmallen
# visualizer.fit_nodes(FirstOrderLINEEnsmallen())
#
# In this tutorial, we use the embedding we have just computed above:

visualizer.fit_nodes(embedding)

# And now we can visualize the node types:
visualizer.plot_node_types()

For the 4D animation we will be rendering a webm,
but most video formats are supported. Analogously to the bidimensional
image, we run:

In [ ]:
visualizer = GraphVisualizer(
    graph,
    n_components=4,
    rotate=True
)
visualizer.fit_nodes(embedding)
visualizer.plot_node_types()

To obtain instead the complete plot with all of the properties visualized,
you may run the following one-liner:

In [ ]:
GraphVisualizer(graph).fit_and_plot_all(embedding)
        

### TensorFlow-based model

In [ ]:
from grape.embedders import FirstOrderLINETensorFlow
embedding = FirstOrderLINETensorFlow().fit_transform(graph)

In [ ]:
from grape import GraphVisualizer
visualizer = GraphVisualizer(graph)

# You can either provide the model name
#visualizer.fit_nodes("First-order LINE", library_name="TensorFlow")
# Or provide a precomputed embedding
#
# visualizer.fit_nodes(numpy_array_with_embedding)
# visualizer.fit_nodes(pandas_dataframe_with_embedding)
#
# or alternatively provide the model to be used:
#
# from grape.embedders import FirstOrderLINETensorFlow
# visualizer.fit_nodes(FirstOrderLINETensorFlow())
#
# In this tutorial, we use the embedding we have just computed above:

visualizer.fit_nodes(embedding)

# And now we can visualize the node types:
visualizer.plot_node_types()

For the 4D animation we will be rendering a webm,
but most video formats are supported. Analogously to the bidimensional
image, we run:

In [ ]:
visualizer = GraphVisualizer(
    graph,
    n_components=4,
    rotate=True
)
visualizer.fit_nodes(embedding)
visualizer.plot_node_types()

To obtain instead the complete plot with all of the properties visualized,
you may run the following one-liner:

In [ ]:
GraphVisualizer(graph).fit_and_plot_all(embedding)
        